# ABC Classification Syncreon Logistica
# A = 85% B = 10% C = 5% D = 0%

In [4]:
import pandas as pd
import csv
import numpy as np
from pandas import *

In [218]:
#geral = open('mes1.csv')
df1 = pd.read_csv("junho.csv", encoding='latin-1', low_memory=False)
df2 = pd.read_csv("julho.csv", encoding='latin-1', low_memory=False)
df3 = pd.read_csv("agosto.csv", encoding='latin-1', low_memory=False)
dfprod = pd.read_csv("products.csv", encoding='latin-1', low_memory=False)

dfgeral = pd.concat([df1,df2,df3])
dfgeral

dfgeral2 = dfgeral.drop(dfgeral.columns[[0,1,2,3,4,5,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,51,52,53]], axis=1)
dfgeral2
dfpivot = dfgeral2.pivot_table(index='PartNo', aggfunc='sum')
dfpivot

dftotal = dfpivot.sort_values(['Qty'], ascending=[0])
dftotal

dfprod.rename(columns={'Material Part No':'PartNo'}, inplace=True)

dftotal2 = pd.merge(dftotal, dfprod[['PartNo','Description']], on='PartNo', how='left')
dftotal2

totalout = dftotal2['Qty'].sum()
#totalout

dftotal2['Percentual'] = (np.true_divide(dftotal2['Qty'], totalout)*100).round(2).astype(float)
#dftotal2['Percentual']
dftotal2
dftotal2['Acomulado'] = np.cumsum(dftotal2['Percentual']).astype(float)
dftotal2
#ok

new_df = dfprod['PartNo']
new_df



new_df = pd.merge(new_df, dfprod[['PartNo','Description']], on='PartNo', how='left')
new_df = pd.merge(new_df, dftotal2[['PartNo','Qty']], on='PartNo', how='left')
new_df = pd.merge(new_df, dftotal2[['PartNo','Percentual']], on='PartNo', how='left')
new_df = pd.merge(new_df, dftotal2[['PartNo','Acomulado']], on='PartNo', how='left')
new_df = pd.merge(new_df, dfprod[['PartNo','Units Per Pack']], on='PartNo', how='left')
new_df = pd.merge(new_df, dfprod[['PartNo','Units Per Pallet']], on='PartNo', how='left')
new_df = pd.merge(new_df, dfprod[['PartNo','Product Category']], on='PartNo', how='left')
new_df = new_df.sort_values(['Percentual'], ascending=[0])

new_df['Curva'] = ''
new_df['Consumption / Day'] = (new_df['Qty'] / 90).round(0)
new_df['Consumption Pall / Day'] = (new_df['Qty'] / new_df['Units Per Pallet']).round(0)

new_df['Qty'].fillna(0, inplace=True)
new_df['Percentual'].fillna(0, inplace=True)
new_df['Acomulado'].fillna(0, inplace=True)

new_df.loc[(new_df.Acomulado <= 85), 'Curva'] = 'A'
new_df.loc[(new_df.Acomulado > 85) & (new_df.Acomulado <= 95), 'Curva'] = 'B'
new_df.loc[(new_df.Acomulado > 95) & (new_df.Acomulado <= 100), 'Curva'] = 'C'
new_df.loc[(new_df.Percentual == 0 ) & (new_df.Acomulado == 0 ), 'Curva'] = 'D'

#delet_specific_value_row
#deletarapc = new_df[ new_df['PartNo'] == 'MQ6G2BR/A' ].index
#new_df.drop(deletarapc , inplace=True)

#new_df


In [209]:
new_df.to_csv('testecurva.csv')